# Analyze runs

# Purpose
Analyze the batch simulations from MLFlow 

# Methodology
* Load data about runs from MLFlow API
* Analyze this data

In [ ]:
# %load ../imports.py
## Local packages:

%matplotlib inline
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False  ## (To fix autocomplete)

## External packages:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)
import numpy as np
np.set_printoptions(linewidth=150)

import numpy as np
import os
import matplotlib.pyplot as plt
#if os.name == 'nt':
#    plt.style.use('presentation.mplstyle')  # Windows

import plotly.express as px 
import plotly.graph_objects as go

import seaborn as sns
import sympy as sp
from sympy.physics.mechanics import (dynamicsymbols, ReferenceFrame,
                                      Particle, Point)
from sympy.physics.vector.printing import vpprint, vlatex
from IPython.display import display, Math, Latex
from vessel_manoeuvring_models.substitute_dynamic_symbols import run, lambdify

import pyro

import sklearn
import pykalman
from statsmodels.sandbox.regression.predstd import wls_prediction_std
import statsmodels.api as sm

from scipy.integrate import solve_ivp

## Local packages:
from vessel_manoeuvring_models.data import mdl

from vessel_manoeuvring_models.symbols import *
from vessel_manoeuvring_models.parameters import *
import vessel_manoeuvring_models.symbols as symbols
from vessel_manoeuvring_models import prime_system
from vessel_manoeuvring_models.models import regression
from vessel_manoeuvring_models.visualization.regression import show_pred
from vessel_manoeuvring_models.visualization.plot import track_plot

## Load models:
# (Uncomment these for faster loading):
import vessel_manoeuvring_models.models.vmm_abkowitz  as vmm 
from vessel_manoeuvring_models.models.vmm import ModelSimulator


In [ ]:
from vessel_manoeuvring_models.models.vmm import ModelSimulator
from vessel_manoeuvring_models.data.mdl import load_test

In [ ]:
from mlflow.tracking.client import MlflowClient
from mlflow.entities import ViewType
from vessel_manoeuvring_models.to_mlflow import tracking_uri
mlflow_client = MlflowClient(tracking_uri=tracking_uri)

In [ ]:
experiment = mlflow_client.get_experiment_by_name(name='VCT_linear')

In [ ]:
experiment.name

In [ ]:
runs = MlflowClient().search_runs(
  experiment_ids=experiment.experiment_id,
  filter_string="",
  #run_view_type=ViewType.ACTIVE_ONLY,
  max_results=1000,
  order_by=["metrics.accuracy DESC"]
)
len(runs)

In [ ]:
df = pd.DataFrame()

for run in runs:
    
    if run.info.status!='FINISHED':
        continue  # Unfinnished simulation
        
    row = pd.Series(run.data.params, dtype='object', name=run.info.run_id)
    row = row.append(pd.Series(run.data.metrics))
    row['status'] = run.info.status
    row['artifact_uri'] = run.info.artifact_uri
    row['experiment_id'] = run.info.experiment_id
    row['experiment'] = experiment.name
    
    df = df.append(row, ignore_index=True)
    
df.index.name='run_id'

In [ ]:
df.head()

In [ ]:
df.sort_values(by='r2', ascending=False).head()

In [ ]:
row = df.sort_values(by='r2', ascending=False).iloc[0]

In [ ]:
from IPython.display import Image
Image(os.path.join(row.artifact_uri.replace(r'file:///',''),'track_plot.png'))


In [ ]:
row

## Load and resimulate

In [ ]:
run_params = row
df, meta_data = load_test(**run_params)

model = ModelSimulator.load(run_params['model'])
result = model.simulate(df)